<a href="https://colab.research.google.com/github/rubanrex/trial_RD/blob/main/RD_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import os
import platform
import subprocess

def check_os():
    os_name = platform.system().lower()
    print(f"Detected OS: {os_name}")
    if os_name == "darwin":
        return "Your OS: Mac\n• Download Docker for Mac: https://www.docker.com/products/docker-desktop\n• Ensure Docker is running.\n• Select the directories and run the program."
    elif os_name == "windows":
        return "Your OS: Windows\n• Download Docker for Windows: https://www.docker.com/products/docker-desktop\n• Ensure Docker is running.\n• Select the directories and run the program."
    elif os_name == "linux":
        return "Your OS: Linux\n• Ensure Docker is installed and running.\n• Select the directories and run the program."
    else:
        return "Unsupported operating system for this script."

def check_docker_running():
    try:
        subprocess.run(["docker", "info"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except subprocess.CalledProcessError:
        return False

def prompt_docker_start():
    while not check_docker_running():
        print("Docker does not seem to be running. Please open Docker Desktop.")
        print("Waiting for Docker to start...")
    print("Docker is now running.")

def find_tar_file(directory):
    tar_files = [f for f in os.listdir(directory) if f.endswith('.tar')]
    if len(tar_files) == 1:
        return os.path.join(directory, tar_files[0])
    elif len(tar_files) > 1:
        return os.path.join(directory, tar_files[0])  # Simplified selection
    else:
        return None

def load_docker_image(tar_path):
    try:
        print(f"Loading Docker image from {tar_path}...")
        subprocess.run(["docker", "load", "-i", tar_path], check=True)
        print("Docker image loaded successfully.")
        return "Docker image loaded successfully."
    except subprocess.CalledProcessError:
        print("Failed to load Docker image. Ensure Docker is installed and the image path is correct.")
        return "Failed to load Docker image. Ensure Docker is installed and the image path is correct."

def run_rd_program(tar_directory, fasta_directory):
    os_check_message = check_os()
    print(os_check_message)
    if "Unsupported" in os_check_message:
        print("Error: Unsupported OS")
        return

    if not os.path.isdir(tar_directory):
        print(f"Directory not found: {tar_directory}")
        return

    if not os.path.isdir(fasta_directory):
        print(f"Directory not found: {fasta_directory}")
        return

    if not check_docker_running():
        prompt_docker_start()

    tar_path = find_tar_file(tar_directory)
    if not tar_path:
        print("No valid .tar file found or selected.")
        return

    load_message = load_docker_image(tar_path)
    if "Failed" in load_message:
        print(load_message)
        return

    os_name = platform.system().lower()
    command = f'''
    for fasta_file in {fasta_directory}/*.fasta; do
        file=$(basename "$fasta_file" .fasta)
        docker run --rm -v {fasta_directory}:/mnt/fasta -e LD_LIBRARY_PATH=/app/RepeatDetector/build/external/htslib/src/htslib repeat-detector /usr/local/bin/RepeatDetecter --prf /app/RepeatDetector/Profiles/CAG/Annex10_cag_correctedFreq_notlog_AND_Complete.prf /mnt/fasta/$file.fasta --output-name "/mnt/fasta/$file.rest" -o histogram --with-revcomp --cycle-range \\[0:500\\] --verbose
    done
    '''

    print("Running RD program...")
    os.system(command)
    print("All .fasta files have been processed successfully.")

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Run RD Program")
    parser.add_argument("--tar-dir", required=True, help="Directory containing the Docker .tar file")
    parser.add_argument("--fasta-dir", required=True, help="Directory containing the FASTA files")

    args = parser.parse_args()
    run_rd_program(args.tar_dir, args.fasta_dir)



usage: colab_kernel_launcher.py [-h] --tar-dir TAR_DIR --fasta-dir FASTA_DIR
colab_kernel_launcher.py: error: the following arguments are required: --tar-dir, --fasta-dir


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
